In [8]:
"""

NOTE: the theorem (https://mathoverflow.net/questions/271932/formula-for-the-frobenius-schur-indicator-of-a-finite-group) only guarantees a bilinear form, not a sesqilinear form.

""";

In [2]:
#define conjugation as x |--> x**q, an order two automorphism of F_q^2. note x**q == x for x \in F_q.
def conjugate_pos_char(A):
    assert A.nrows() == A.ncols()
    field_size = A.base_ring().order()
    q = sqrt(field_size) if field_size.is_square() else field_size
    return matrix(GF(q**2),[[A[i][j]**q for j in range(A.nrows())] for i in range(A.nrows())])

In [42]:
def invariant_symmetric_sesquilinear_matrix(q,partition):
    """
    Computes the matrix of a S_n-invariant symmetric sesquilinear form.

    Sets up and solves system of linear equations based on writing U as an unknown in polynomial ring generators. 

    The equations are \rho(g)U = \lambda_g*U*\rho(g)^{-1 T} where \lambda_g = \det(\rho(g))\overline{\det(\rho(g))}^T.

    The variables for U can be extracted to yield a matrix over GF(q^2) for each g. 
    
    These are stacked to get the overall system, and we find the one dim'l null space to get a solution vector, and format as a matrix.

    Note: one could also form the Kroenecker products \rho(g) \otimes \rho(g)^{-1 T} to explicitly obtain the system.
    
    """

    # Define the group G and its rep'n as a Specht module, dimension
    n = sum(partition)
    SGA = SymmetricGroupAlgebra(GF(q^2), n)
    SM = SGA.specht_module(partition)
    G = SGA.group()
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    
    # Initialize U as a matrix of variables over GF(q^2)
    R = PolynomialRing(GF(q^2), 'u', d_rho^2)
    U_vars = R.gens()  # List of variable generators for U
    U = Matrix(R, d_rho, d_rho, U_vars)  # U is a d_rho x d_rho matrix of variables
    
    # for each generator of G, form the augmented system 
    def augmented_matrix(g):
    
        rho_g = rho(Permutation(g))
        rho_g_conj_inv_T = conjugate_pos_char(rho_g.inverse().transpose())
    
        # Compute lambda_g
        det_rho_g = det(rho_g)
        lambda_g = det_rho_g * (det_rho_g ** q)
    
        # Form the matrix equation
        equation_matrix = rho_g * U - lambda_g * U * rho_g_conj_inv_T
    
        # Initialize a list to hold rows of the augmented system
        augmented_system = []
    
        # Extract coefficients for each linear equation in the matrix
        for i in range(d_rho):
            for j in range(d_rho):
                # Get the (i, j) entry of the equation matrix, which is a linear combination of the u variables
                linear_expression = equation_matrix[i, j]
            
                # Extract the coefficients of each u_k in the linear expression
                row = [linear_expression.coefficient(u) for u in U_vars]
            
                # Append the row to the augmented system
                augmented_system.append(row)
    
        # Convert the augmented system to a matrix
        return Matrix(GF(q^2), augmented_system)
    
    total_system = matrix(GF(q^2),0,d_rho^2)
    for g in G:
        total_system = total_system.stack(augmented_matrix(g))
    
    #compute the null space of the overall matrix
    null_space = total_system.right_kernel()
    
    #return a d_rho x d_rho matrix over GF(q^2) from the 1 dim'l null space given as vector
    return matrix(GF(q^2),d_rho,d_rho,null_space.basis()[0])

In [11]:
def sesquilinear_form(x,y,U):
    return x*U*y.transpose()

In [80]:
#define the field size and partition labeling rep'n
q=3; partition=[3,2]

[ 1  1  1  0 -1]
[ 1  0  1 -1 -1]
[ 1  1  0 -1 -1]
[ 0 -1 -1  0 -1]
[-1 -1 -1 -1  0]

In [104]:
#ensure the resulting form is G-invariant, symmetric, bi/sesquilinear
def check_form_properties():
    #define the representation matrix corresponding to q, partition
    SGA = SymmetricGroupAlgebra(GF(q^2),sum(partition))
    SM = SGA.specht_module(partition)
    rho = SM.representation_matrix
    d_rho = SM.dimension()
    G = SGA.group()

    #define variables as polynomial generators
    R_xy = PolynomialRing(GF(q^2), d_rho, var_array='x,y')
    x = matrix([R_xy.gens()[2*i] for i in range(d_rho)])
    y = matrix([R_xy.gens()[2*i+1] for i in range(d_rho)])
    R_xy_lambda = PolynomialRing(R_xy,'lambda')
    lambda_ = R_xy_lambda.gens()[0]

    #compute the bilinear form matrix
    
    #check symmetric property
    symmetric = sesquilinear_form(x,y,U_form) == sesquilinear_form(y,x,U_form)
    print("symmetric=" + str(symmetric))
    #check G-invariance property
    G_invariant = all(sesquilinear_form((rho(g)*x.transpose()).transpose(),(rho(g)*y.transpose()).transpose(),U_form) == sesquilinear_form(x,y,U_form) for g in G)
    print("G-invariant=" + str(G_invariant))
    #check sesquilinear property
    first_arg = sesquilinear_form(lambda_*x,y,U_form) == lambda_*sesquilinear_form(x,y,U_form)
    second_arg = sesquilinear_form(x,lambda_*y,U_form) == lambda_*sesquilinear_form(x,y,U_form) #need to amend for conjugation
    print("bilinear=" + str(first_arg and second_arg))

    return symmetric and G_invariant and first_arg and second_arg

In [105]:
check_form_properties()

symmetric=True
G-invariant=False
bilinear=True


False